# World Happiness Report

by Leokobi

This analysis was conducted based on data provided by the "Sustainable Development Solutions Network" at Kaggle. (More information https://www.kaggle.com/unsdsn/world-happiness). For this analysis, the 5 years of data set available from 2015 to 2019 were analyzed. The data comes from researches carried out in different countries on the population's perception about different social aspects of their land. The objective of this notebook is the processing of data, data visualization and how the perception of social factors influence the level of happiness.


Meaning of the columns:

Country-Country name

Region- Region in which the country was categorized.

Happiness Rank - Classification of the country in a given year.

Happiness Score 

Economy - Quantification of economic power.

Government Trust - A quantification of people's perceived trust in their governments.

Generosity- Estimated numerical value based on the perception of generosity experienced by respondents in their country.

Social Support- Metric that estimates people's satisfaction with their friends and family.

Freedom- Perception of quantified freedom.


# Importing Libraries

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import glob as glob
import geopandas as gpd
from geopy.geocoders import Nominatim
import pycountry
import os
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.stattools import durbin_watson



# Data Cleansing  and  Preprocessing

In [ ]:
#On Kaggle Notebook
all_files=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        all_files.append(os.path.join(dirname, filename))

    
#Reorder
myorder = [2, 3, 0, 4, 1]
all_files = [all_files[i] for i in myorder]


In [ ]:
# #Downloading files  from Kaggle and unziping in folder "Dados"
# !kaggle datasets download -d unsdsn/world-happiness

# with zipfile.ZipFile(os.getcwd()+'\world-happiness.zip', 'r') as zip_ref:
#     zip_ref.extractall(os.getcwd()+'\Dados')
# #Selectig  all .csv files    
# all_files = glob.glob(os.getcwd()+'\Dados' + "/*.csv")

In [ ]:
#Extracting file names
year=[os.path.basename(yr)[:4] for yr in all_files]

li = []
i=0
for filename in all_files:
    
    df_temp = pd.read_csv(filename, index_col=None, header=0)
    df_temp['Year']=year[i]
    li.append(df_temp)
    i+=1
    
#Removing some columns so that all files have the same information.

df_2015=li[0].drop(['Standard Error','Dystopia Residual'], axis=1)
df_2016=li[1].drop(['Lower Confidence Interval','Upper Confidence Interval','Dystopia Residual'], axis=1)
df_2017=li[2].drop(['Whisker.high','Whisker.low','Dystopia.Residual'], axis=1)
df_2018=li[3]
df_2019=li[4]



#Renaming columns
df_2015.rename(columns={'Economy (GDP per Capita)':'Economy', 'Health (Life Expectancy)':'Life Expectancy',
        'Trust (Government Corruption)':'Government Trust','Family':'Social Support'},inplace=True)

df_2016.rename(columns={'Economy (GDP per Capita)':'Economy', 'Health (Life Expectancy)':'Life Expectancy',
        'Trust (Government Corruption)':'Government Trust','Family':'Social Support'},inplace=True)

df_2017.rename(columns={'Happiness.Rank':'Happiness Rank', 'Happiness.Score':'Happiness Score',
        'Economy..GDP.per.Capita.':'Economy', 'Health..Life.Expectancy.':'Life Expectancy',
        'Trust..Government.Corruption.':'Government Trust','Family':'Social Support'},inplace=True)

df_2018.rename(columns={'Overall rank':'Happiness Rank', 'Country or region':'Country', 'Score':'Happiness Score', 
                        'GDP per capita':'Economy','Social support':'Social Support', 
                        'Healthy life expectancy':'Life Expectancy', 'Freedom to make life choices':'Freedom',                        
                        'Perceptions of corruption':'Government Trust'},inplace=True)

df_2019.rename(columns={'Overall rank':'Happiness Rank', 'Country or region':'Country', 'Score':'Happiness Score', 
                        'GDP per capita':'Economy','Social support':'Social Support', 
                        'Healthy life expectancy':'Life Expectancy', 'Freedom to make life choices':'Freedom',                        
                        'Perceptions of corruption':'Government Trust'},inplace=True)


#Mapping regions not available in other years.

Country_Region=df_2015.set_index('Country')['Region']

df_2017['Region']=df_2017['Country'].map(Country_Region)
df_2018['Region']=df_2018['Country'].map(Country_Region)
df_2019['Region']=df_2019['Country'].map(Country_Region)

df_t=[df_2015,df_2016,df_2017,df_2018,df_2019]


In [ ]:
# Checking for null values

[print('Year',i+2015 , '\n' ,df_t[i].isnull().any()) for i in range(len(list(df_t)))]

In [ ]:
#Countries without Regions, countries that were not at df_2015
print('Countries without Region 2017:',df_2017[df_2017['Region'].isnull()]['Country'].index,
list(df_2017[df_2017['Region'].isnull()]['Country']))
print('Countries without Region 2018:',df_2018[df_2018['Region'].isnull()]['Country'].index,
list(df_2018[df_2018['Region'].isnull()]['Country']))
print('Countries without Region 2019:',df_2019[df_2019['Region'].isnull()]['Country'].index,
list(df_2019[df_2019['Region'].isnull()]['Country']))

In [ ]:
#Adding Regions to these Countries
df_2017.loc[list(df_2017[df_2017['Region'].isnull()]['Country'].index),'Region']=['Eastern Asia','Latin America and Caribbean','Eastern Asia','Sub-Saharan Africa','Sub-Saharan Africa','Sub-Saharan Africa']
df_2018.loc[list(df_2018[df_2018['Region'].isnull()]['Country'].index),'Region']=['Latin America and Caribbean','Latin America and Caribbean','Middle East and Northern Africa','Sub-Saharan Africa','Sub-Saharan Africa','Sub-Saharan Africa']
df_2019.loc[list(df_2019[df_2019['Region'].isnull()]['Country'].index),'Region']=['Latin America and Caribbean','Middle East and Northern Africa','Central and Eastern Europe','Sub-Saharan Africa','Sub-Saharan Africa','Sub-Saharan Africa','Sub-Saharan Africa']


In [ ]:
#Verifying null value in 2018

df_2018[df_2018['Government Trust'].isnull()]

In [ ]:
#Government confidence values for other years

for j in range(len(df_t)):
    print('Year:',j+2015,df_t[j][df_t[j]['Country']=='United Arab Emirates']['Government Trust'])

In [ ]:
#Setting value in 2018 as the average between 2017 and 2019
df_2018.loc[list(df_2018[df_2018['Government Trust'].isnull()].index),'Government Trust']=list((df_2017[df_2017['Country']=='United Arab Emirates']['Government Trust']+df_2019[df_2019['Country']=='United Arab Emirates']['Government Trust'])/2)

In [ ]:
#Concateating Dataframes
df = pd.concat(df_t, axis=0, ignore_index=True)

#Renaming some countries, necessary for future analysis in Geopandas and Pycountry
df.replace(['Congo (Brazzaville)', 'Congo (Kinshasa)','Swaziland','South Korea','Laos','Ivory Coast','Trinidad & Tobago','Hong Kong S.A.R., China','Taiwan Province of China','Niger'], 
           ['Congo, The Democratic Republic of the','Congo','Eswatini','Korea, Republic of','Lao',"Côte d'Ivoire",'Trinidad and Tobago','Hong Kong Special Administrative Region of China','Taiwan, Province of China','Republic of the Niger'],inplace=True)

#Removing unrecognized countries
remove = ['North Cyprus','Somaliland region','Swaziland','Palestinian Territories','Somaliland Region','Northern Cyprus']
for r in remove:
    df.drop(list(df[df["Country"]==r].index), inplace=True)   


In [ ]:
#Mapping missing regions
Country_Region_C=df.drop_duplicates('Country',keep='first')[['Country','Region']].set_index('Country')


# Creating GeoDataFrame

In [ ]:
## Way to find Latitude and Longitude

# geolocator = Nominatim(user_agent="My_app")
# country_location=[]
# for country  in  df['Country'].unique():
#     try:
#         location = geolocator.geocode(country)
#         a=[country,location.latitude, location.longitude]
#         country_location.append(a)
#     except:
#         pass
    

In [ ]:
#Finding Iso Alpha3 Code for countries, e.g. Brazil = BRA

i=0
country_location=[]
for c in df['Country'].unique():
    try:
        alph3=pycountry.countries.search_fuzzy(c)[0].alpha_3
        a=[c,alph3]
        
        country_location.append(a)
    except:
        pass
    i+=1
    

In [ ]:
#Checking if all countries have been found

nf=[]
for c in df['Country'].unique():
     if c in [cl[0] for cl  in country_location]:
         pass
     else:
         print('Country not found:',c)
         nf.append(c)

In [ ]:
# Scaling data
scaled_data=StandardScaler().fit_transform(df.loc[:,'Happiness Score':'Generosity'])
df_scale=pd.DataFrame(scaled_data,columns=df.loc[:,'Happiness Score':'Generosity'].columns)
df_scale['Year']=df['Year'].reset_index(drop=True)
df_scale['Region']=df['Region'].reset_index(drop=True)
df_scale['Country']=df['Country'].reset_index(drop=True)


In [ ]:
#Adding alph3 column to dataframe and grouping by country
df_cl=pd.DataFrame(country_location,columns=['Country','Alpha3'])
df_cl.set_index('Country',inplace=True)
df_c=df_scale.groupby(df['Country']).mean()
df_c['Alpha3']=df_c.index.map(df_cl['Alpha3'])



In [ ]:
#Transforming dataframe to geodataframe

gdf = gpd.GeoDataFrame(df_c).reset_index()
gdf['Region']=gdf['Country'].map(Country_Region_C['Region'])


# Plotting choropleth maps


As the data was grouped by country, the graphs below are plotted based on the average values for the years 2015 to 2019. Countries without data are grayed out.


In [ ]:
#Accessing geo map geopandas database
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world=world[(world['name']!="Antarctica")]

#Some alpha iso3 codes missing from basemap
world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
world.loc[world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'RKS'

In [ ]:
#Mapping geometric data in the Geodataframe
gdf['geometry']=gdf['Alpha3'].map(world.drop_duplicates(['iso_a3']).set_index(['iso_a3'])['geometry'])

In [ ]:
#Happines Score map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='grey')
gdf.plot(ax=base,column='Happiness Score',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5)
ax.set_title("Happiness Score",fontsize=20)
ax.set_axis_off()

In [ ]:
#Economy map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Economy',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5 )
ax.set_title("Economy",fontsize=20)
ax.set_axis_off()

In [ ]:
#Life Expectancy map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Life Expectancy',legend=True,cmap='jet_r' ,edgecolor='white',linewidth=0.5)
ax.set_title("Life Expectancy",fontsize=20)
ax.set_axis_off()

In [ ]:
#Freedom map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Freedom',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5 )
ax.set_title("Freedom",fontsize=20)
ax.set_axis_off()

In [ ]:
#Generosity map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Generosity',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5 )
ax.set_title("Generosity",fontsize=20)
ax.set_axis_off()

In [ ]:
#Social Support map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Social Support',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5 )
ax.set_title("Social Support",fontsize=20)
ax.set_axis_off()

In [ ]:
#Government Trust map
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Government Trust',legend=True,cmap='jet_r',edgecolor='white',linewidth=0.5 )
ax.set_title("Government Trust",fontsize=20)
ax.set_axis_off()

# Exploratory analysis


In [ ]:
#Region Divisions
fig, ax = plt.subplots(figsize=(25,7))
base = world.plot(ax=ax,color='grey', edgecolor='white')
gdf.plot(ax=base,column='Region',cmap='jet_r',edgecolor='white',linewidth=0.5,legend=True, legend_kwds={'loc': 'lower left'} )
ax.set_title("Regions Division",fontsize=20)
ax.set_axis_off()

In [ ]:
#Country % ranking for each feature and year
for yr in list(df_scale['Year'].unique()):
    for col in df_scale.columns[0:7]:
        df_scale.loc[df_scale['Year'] == yr,str('Ranking '+col)]=  round(df_scale.loc[df_scale['Year'] == yr, col].rank(method='dense',ascending=False)/df_scale.loc[df_scale['Year'] == yr, col].count()*100,2)

In [ ]:
#DataFrame description
display(df_scale.describe())
#Boxplot
fig, ax = plt.subplots(figsize=(5,5))
sns.boxplot(data=df_scale.iloc[:,0:7])
plt.xticks(rotation=25,ha='right');
ax.set_title("Distribuição das variáveis",fontsize=20)

It is noticed the presence of some outliers, mainly in the Government Trust and Generosity categories in their upper ranges, meaning that only some countries have indicators above the majority in these indicators.

In [ ]:
# Visualization of the level of Happiness by year and by region

fig, ax = plt.subplots(figsize=(25,7))
sns.barplot(x=df_scale['Region'],y=df_scale['Happiness Score'],hue=df_scale['Year'])
ax.set_title("Happiness Score by Region and Year",fontsize=20)
plt.xticks(rotation=25,ha='right',fontsize=15);

The Australia and New Zealand region averages the highest happiness value, while the Sub-Saharan Africa region the lowest. The region of North America and Latin America and the Caribbean showed an average regression of the level of happiness in these years.

In [ ]:
# Brazil

brazil=df_scale.loc[df_scale['Country']=='Brazil','Year':'Ranking Generosity']
fig, ax = plt.subplots(figsize=(10,8))
brdf = pd.melt(brazil.iloc[:,[0,3,4,5,6,7,8,9]],id_vars='Year')
sns.barplot(data=brdf,x='variable',y='value',hue='Year')
plt.ylabel('Brazil ranking %')
plt.xlabel('Variables')
ax.set_title("Brazil's Ranking for each variable",fontsize=20)
plt.xticks(rotation=25,ha='right');



Between the years 2015 and 2019 Brazil has consecutively lost positions in the ranking of Happiness, as well as in Government Trust. Economically and in Social Support indicators, it remained in similar positions. There was an improvement in the life expectancy and generosity indicator.
Although the social indicators in Brazil are not so good, the levels of Happiness are high.

In [ ]:
#Verifying data distribution
fig = plt.figure(figsize=(15,35))
i=0
for reg in list(df['Region'].unique()):  
    ax=plt.subplot(5,3,i+1)
    
    sns.boxplot(data=df_scale[df_scale['Region']==reg].iloc[:,0:7])
    
    ax = sns.stripplot(data=df_scale[df_scale['Region']==reg].iloc[:,0:7],
                   color="black", edgecolor="gray")
    
    t=str(reg+' / n observações= '+str(df_scale.groupby('Region').count().loc[reg][0]))
    ax.set_title(t,fontsize=11)
    plt.xticks(rotation=25,ha='right');
    i+=1
plt.subplots_adjust(hspace=0.3,wspace=0.5)

In [ ]:
#Pair plot
plt.figure(figsize=(5,5));
sns.pairplot(df_scale.iloc[:,0:7],diag_kind='kde',kind='reg',plot_kws={'line_kws':{'color':'red'}});

There is a clear linearity trend between happiness and economic indicators, social support and life expectancy,also between life expectancy and economy
There is an imbalance in the distribution of confidence in the government, with a tail to the right, that is, few countries have high values in this indicator.
Countries with a high degree of confidence in the Government tend, with the exception of the Generosity indicator, to present very high social indicators.

In [ ]:
#Correlation map
fig= plt.figure(figsize=(10,10))

ax1=plt.subplot(2,1,1)

sns.heatmap(df_scale.iloc[:,0:7].corr(), annot=True, fmt=".2f", linewidths=.5,vmin=-1, vmax=1, center= 0,ax=ax1)  

Analyzing the correlation numerically, it is observed that the level of Happiness is more correlated with the Economy and Life Expectancy than with Generosity, Freedom and Government Trust. Note: correlation does not imply in causality.


# Regression

In this section, two models of multiple linear regrassion will be implemented, Ordinary Least Squares from the stats model package and the stochastic gradient descent from skleanr package.


In [ ]:
#Scaled data X y split
SEED=np.random.seed(401)
Xs=df_scale.iloc[:,1:7]

ys=df_scale.iloc[:,0]

##Para estrafificar dados  continuos
# #Regra de Sturger qquantidade de bins
# nbins=np.around(1+10/3*np.log10(ys_pre.count())).astype(int)
# #divisao dos bins
# bins=pd.cut(ys_pre,nbins,labels=np.linspace(1,nbins,nbins))
# bins.name='bins'
# ys=pd.concat([ys_pre,bins],axis=1)

#Train Test Split
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys,test_size=0.30,random_state=SEED)
#Adding constant for stats model package
Xsc_train = sm.add_constant(Xs_train)
Xsc_test = sm.add_constant(Xs_test)

In [ ]:
#Data train test distribuition
fig= plt.figure(figsize=(14,7))
plt.subplots_adjust(hspace=0.5,wspace=0.5)    

ax1=plt.subplot(1,3,1)
ax1.set_title('Df dist tamanho:'+str(len(ys)))
sns.distplot(ys,ax=ax1,kde=True,hist=True);

ax2=plt.subplot(1,3,2)
ax2.set_title('Train dist tamanho:'+str(len(ys_train)))
sns.distplot(ys_train,ax=ax2,kde=True,hist=True);

ax3=plt.subplot(1,3,3)
ax3.set_title('Test dist tamanho:'+str(len(ys_test)))
sns.distplot(ys_test,ax=ax3,kde=True,hist=True);

In [ ]:
#Verifying Multicollinearity of the data 
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=pd.Series([variance_inflation_factor(Xs.values,i) for i in range(Xs.shape[1])], index=Xs.columns)
print(VIF)

If the value is higher than 5 then the explanatory variable is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

In [ ]:
#Ordinary Least Squares model
mOLS_scale=sm.OLS(ys_train,Xsc_train)
lr_scale=mOLS_scale.fit()
display(lr_scale.summary())

#Residuals
predict_train=lr_scale.predict(Xsc_train)
residual_train=lr_scale.resid
predict_test=lr_scale.predict(Xsc_test)
residual_test=ys_test-predict_test
residual_test_OLS=residual_test

#R2 square
OLS_r2_train=sk.metrics.r2_score(ys_train,predict_train)
OLS_r2_test=sk.metrics.r2_score(ys_test,predict_test)

#Mean Square error
OLS_MSE=mean_squared_error(ys_test,predict_test)

#test durbin_watson statistic for n=550 and k=6 dl=1.780 dU=1.824 H0= null hypothesis of zero autocorrelation o 
#if dw>dU do not reject
dw=durbin_watson(residual_train)

#H0=all observations have the same error variance, the errors are homocedastic
bp_scale=diag.het_breuschpagan(residual_test, Xsc_test)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']

#Dataframe with coeficient and parameters
OLS_coef=lr_scale.params.tolist()
label_coef=[['Coef'],['const','Economy','Social Support','Life Expectancy','Freedom','Government Trust','Generosity']]
idx_c=pd.MultiIndex.from_product(label_coef,names=['Type','Name'])
OLS_c=pd.Series(OLS_coef,name='OLS',index=idx_c)

OLS_eval=[OLS_r2_train, OLS_r2_test,OLS_MSE,bp_scale[1],dw]
label_ev=[['Eval'],['R2 Treino', 'R2 Teste', 'MSE Teste', 'p-value Heterocedasticidade','Durbin Watson']]
idx_e=pd.MultiIndex.from_product(label_ev,names=['Type','Name'])
OLS_e=pd.Series(OLS_eval,name='OLS',index=idx_e)

OLS_df=pd.DataFrame(pd.concat([OLS_e,OLS_c],axis=0))
display(OLS_df)

#Plot of the predicted value and squared error against real value
fig= plt.figure(figsize=(20,5))

ax1=plt.subplot(1,2,1)
ax1.set_title('Real x Pred (Blue=Train, Green=Test)',fontsize=15)

plt.scatter(x=ys_train,y=predict_train,c='b');
plt.scatter(x=ys_test,y=predict_test,c='g');


ax2=plt.subplot(1,2,2)
ax2.set_title('Real x Resid² (Blue=Train, Green=Test)',fontsize=15)
plt.scatter(x=ys_train,y=residual_train**2,c='b');
plt.scatter(x=ys_test,y=residual_test**2,c='g');


In [ ]:
#distribuição  dos  residuos
sns.distplot(residual_train,kde=True,hist=True)
sm.qqplot(residual_test,line='s')
plt.show()

In [ ]:
#Stochastic gradient descent
mSGD=SGDRegressor(loss='squared_loss')
mSGD.fit(Xs_train,ys_train)

#Model residual and prediciton
predict_train=mSGD.predict(Xs_train)
residual_train=ys_train-predict_train

predict_test=mSGD.predict(Xs_test)
residual_test=ys_test-predict_test
residual_test_SGD=residual_test

#R2 square
SGD_r2_train=sk.metrics.r2_score(ys_train,predict_train)
SGD_r2_test=sk.metrics.r2_score(ys_test,predict_test)

#Mean Square error
mSGD_MSE=mean_squared_error(ys_test,predict_test)


#Verificar Heterostacidade p>0.05 para regeitar hHeterostacidade
bp_scale=diag.het_breuschpagan(residual_test,Xsc_test)

#durbin_watson Test,for n=550 and k=6  => dl=1.780 dU=1.824; H0=null hypothesis of zero if dw>dU do not reject
dw=durbin_watson(residual_train)

#Dataframe with coeficient and parameters
mSGD_coef= mSGD.coef_
mSGD_intercept=mSGD.intercept_
mSGD_param=np.concatenate([mSGD_intercept,mSGD_coef])
label_coef=[['Coef'],['const','Economy','Social Support','Life Expectancy','Freedom','Government Trust','Generosity']]
idx_c=pd.MultiIndex.from_product(label_coef,names=['Type','Name'])
SGD_c=pd.Series(mSGD_param,name='SGD',index=idx_c)

SGD_eval=[SGD_r2_train, SGD_r2_test, mSGD_MSE, bp_scale[1],dw]
label_ev=[['Eval'],['R2 Treino', 'R2 Teste', 'MSE Teste', 'p-value Heterocedasticidade', 'Durbin Watson']]
idx_e=pd.MultiIndex.from_product(label_ev,names=['Type','Name'])
SGD_e=pd.Series(SGD_eval,name='SGD',index=idx_e)

SGD_df=pd.DataFrame(pd.concat([SGD_e,SGD_c],axis=0))
display(SGD_df)

#Plot of the predicted value and squared error against real value
fig= plt.figure(figsize=(20,5)) 

ax1=plt.subplot(1,2,1)
ax1.set_title('Train Real x Pred (Blue=Train, Green=Test)',fontsize=15)

plt.scatter(x=ys_train,y=predict_train,c='b');
plt.scatter(x=ys_test,y=predict_test,c='g');


ax2=plt.subplot(1,2,2)
ax2.set_title('Train Real x Resid² (Blue=Train, Green=Test)',fontsize=15)
plt.scatter(x=ys_train,y=residual_train**2,c='b');
plt.scatter(x=ys_test,y=residual_test**2,c='g');


In [ ]:
sns.distplot(residual_train,kde=True,hist=True)
sm.qqplot(residual_test,line='s')
plt.show()

By the values of R2, it is posssible to say that both the OLS and SGD models can explain 75.9 and 77.7% of the variations of the input data, for the training and test data respectively 

The p-value for all model explanatory variables showed a value less than .05, indicating that they are highly significant for the model

The Durbin Watson test was larger than the upper limit, so the null autocorrelation hypothesis cannot be rejected.

Observing the residual2 dispersion grahp, we can infer that visually the model tends to present more significant errors for countries that have a lower level of happiness.

As in the Breusch-Pagan test the p-value is greater than .15 on OLS and greater than 0.13 on SGD, so there is little statistical significance against the acceptance of the null hypothesis of homeostacy.

The histogram shows that both the models tends to overestimate the happiness value, given the larger tail in the negative values

visually by q-q graph the residues approach a normal

Thus the model meets the Four Assumptions of Linear Regression:

Linear relationship: There exists a linear relationship between the independent variable, x, and the dependent variable, y.

Independence: The residuals are independent. In particular, there is no correlation between consecutive residuals in time series data.

Homoscedasticity: The residuals have constant variance at every level of x.

Normality: The residuals of the model are normally distributed.


In [ ]:
fig= plt.figure(figsize=(20,5)) 

ax=plt.subplot(1,2,1)
ax.set_title('Test Real x Residual2 OLS Vs. SGD (Blue=OLS, Red=SGD)',fontsize=15)
plt.scatter(x=ys_test,y=residual_test_OLS**2,c='b',marker='.');
plt.scatter(x=ys_test,y=residual_test_SGD**2,c='r',marker='.',alpha=0.5);


In [ ]:
comp=pd.concat([OLS_df,SGD_df],axis=1)
display(comp)
comp.loc['Eval'].plot(kind='bar')
plt.xticks(rotation=25,ha='right');
comp.loc['Coef'].plot(kind='bar')
plt.xticks(rotation=25,ha='right');

Comparing the performance of  both models they are very similar.
Since the modols where feeded by scalled data its possible to  infer that the variable's coeficient is comperable to the wheight, the importance, the model gives to each explanatory variable. Both models tend to give a higher wheight for the Economy folowed by Life Expectancy indicator, and the least wheight for Generosity and Government Trust. This align with the correlation analysis done previously.

In [ ]:
#Extracting those with residual above 2 sig.
resd=residual_train
sig2sup=residual_train.std()*2+residual_train.mean()
sig2inf=-residual_train.std()*2+residual_train.mean()

bottom=resd[resd<sig2inf]
top=resd[resd>sig2sup]
idx=(top+bottom).index

df_tb=df_scale.loc[idx,:]
df_tb['Residual']=pd.concat([top,bottom])
df_tb['tb']='Sobre Estimados'
for ind in top.index:
    df_tb.loc[ind,'tb']='Sub Estimados'
    
df_tb.sort_values('tb',inplace=True)       


In [ ]:
#Verifying data distribution for each year
fig = plt.figure(figsize=(20,60))
i=0
n=df_tb.count()[0]
for ind in list(df_tb.index):  
    ax=plt.subplot(int(np.ceil(n/3)),3,i+1)
    ax.set_ylim([0,100])
    
    df_tb.loc[ind].iloc[10:17].plot(kind='bar', colormap='Blues_r')
    
    t=str(df_tb.loc[ind].iloc[9]+' '+df_tb.loc[ind].iloc[18]+' Residual: '+
          str(round(df_tb.loc[ind].iloc[17],2))+' Year: '+df_tb.loc[ind].iloc[7])
    ax.set_title(t,fontsize=11)
    plt.xticks(rotation=45,ha='right');
    i+=1
plt.subplots_adjust(hspace=1,wspace=0.5)

Analyzing the datas that have more than two sigmas of error, it can be seen that part of the countries that had their happiness value overestimated, that is, that were predicted to be happier than they should have, present at least one of the following characteristics:
-Country has one or two of its indicators way above the other, Rwanda 2017 2016 2018 and 2019 and Tanzania 2017 presents very high indicators of freedom and government confidence, represented graphically by the low position on the ranking.
-Countries that have factors not evaluated by the survey, e.g. HongKong which despite presenting high indicators has a relatively low level of Happiness, very likely due to the political issues of the Region.

For countries that are underestimated, that is, where they were predicted with a higher happiness value than in reality, it is very likely that there is another factor not explained by the data, since the average values of the indicators are always well above the true happiness. Another hypothesis is that the weight given to social indicators may be different from that established by the models, the Economy of these countries may not be the most important factor in Happiness